In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import sys 
sys.path.append('../warpspeed')

from predict_metrics import *
import sympy as sp
from warpspeedkernel import *
import matplotlib.pyplot as plt

In [ ]:
blockSize = (512, 1, 1)

device = DeviceMI210()
preds = {}
imageCount = 8

In [ ]:
imageSize = (1024, 1024)



for inputChannels in range(1,8):
    for outputChannels in range(1, 8):
        print(str(inputChannels) + " x " + str(outputChannels))
        #if (inputChannels, outputChannels) in preds:        
        #    continue
            
        loadFields = []
        storeFields = []

        imageLoads = []

        stencilLoads = []
        for x in range(3):
            for y in range(3):
                for outputChannel in range(outputChannels):
                    stencilLoads.append((  str(x), str(y), str(outputChannel) ))
                    
        #loadFields.append(Field(name = "stencil",
        #                        addresses = stencilLoads,
        #                        datatype = 4,
        #                        dimensions = (3,3,outputChannels),
        #                        alignment = 0))
        
        
        
        for imageChannel in range(inputChannels):
            for x in [-1,0,1]:
                for y in [-1, 0, 1]:
                    imageLoads.append(("tidx + " + str(x), "tidy + " + str(y), str(imageChannel) ))


        loadFields.append(Field(name = "images",
                                addresses = imageLoads,
                                datatype = 4,
                                dimensions = (*imageSize, inputChannels),
                                alignment = 0,
                                multiplicity=imageCount))


        outputStores = []
        for outputChannel in range(outputChannels):
            outputStores.append(("tidx", "tidy", str(outputChannel) ))


        storeFields.append(Field(name = "output",
                                 addresses = outputStores,
                                 datatype = 4,
                                 dimensions = (*imageSize, outputChannels),
                                 alignment = 0,
                                 multiplicity=imageCount))



        kernel = WarpspeedKernel(loadFields, storeFields, registers = 128, flops = imageCount*inputChannels*outputChannels*2)

        lc = LaunchConfig.compute(kernel, blockSize, (imageSize[0]-2, imageSize[1]-2, 1), (1,1,1), device, 0)
        basic = BasicMetrics.compute(lc, device, kernel)
        pred = DerivedMetrics(lc, basic, device)

        print(basic)
        print(pred)
        preds[(inputChannels, outputChannels)] = pred


In [ ]:
fig,ax = plt.subplots(figsize=(8,4), dpi=100)

values = np.zeros((8,8))

for inputChannel, outputChannel in preds.keys():
       values[inputChannel][outputChannel] = preds[(inputChannel, outputChannel)].perfV3*preds[(inputChannel, outputChannel)].lc.flops


        

plt.imshow(values, origin="lower")
ax.set_xlabel("input channels")
ax.set_ylabel("output channels")

ax.set_xlim((0.5,7.5))
ax.set_ylim((0.5,7.5))
cb = plt.colorbar()
cb.set_label("GFlop/s")
plt.show()